In [44]:
%matplotlib inline
import plot
import market
import metrics
from decimal import Decimal
import withdrawal
import harvesting

import pandas
import seaborn
from matplotlib import pyplot as plt
import matplotlib

import collections
import datetime
from dateutil.relativedelta import relativedelta
import scipy.stats
from statsmodels.formula.api import ols

from icecream import ic

In [10]:
seaborn.set(style='whitegrid')
seaborn.set_context('poster')

In [28]:
def str_to_date(s):
    (year, month) = s.split('.')
    def convert_month(m):
        if m == '01':
            return 1
        elif m == '1':
            return 10
        else:
            return int(m)
    return datetime.date(int(year), convert_month(month), 1)
shiller = pandas.read_csv('shiller_monthly.csv', converters={'Date': str_to_date}, index_col='Date')
shiller.head()

,S&P Price,S&P Dividend,S&P Earnings,CPI,Long Rate,Real Price,Real Dividend,Real Earnings,CAPE10,Mean,Median,MoM Price,Div Yield,MoM TR,TR+1,3 month TR,6 month TR,12 month TR,24 month TR,36 month TR
Date,,,,,,,,,,,,,,,,,,,,
1871-01-01,4.44,0.26,0.4,12.46,5.32,85.84,5.03,7.73,NaN,NaN,NaN,NaN,0.004880,NaN,NaN,0.082351,0.094767,0.155934,0.285545,0.255046
1871-02-01,4.50,0.26,0.4,12.84,5.32,84.42,4.88,7.50,NaN,NaN,NaN,0.013514,0.004815,0.018328,1.018328,0.094517,0.093562,0.145034,0.278622,0.276545
1871-03-01,4.61,0.26,0.4,13.03,5.33,85.22,4.81,7.39,NaN,NaN,NaN,0.024444,0.004700,0.029144,1.029144,0.059548,0.078442,0.154054,0.239044,0.229517
1871-04-01,4.74,0.26,0.4,12.56,5.33,90.94,4.99,7.67,NaN,NaN,NaN,0.028200,0.004571,0.032771,1.032771,0.011471,-0.004785,0.153328,0.189443,0.164901
1871-05-01,4.86,0.26,0.4,12.27,5.33,95.41,5.10,7.85,NaN,NaN,NaN,0.025316,0.004458,0.029775,1.029775,-0.000872,-0.018520,0.125026,0.163252,0.108653


In [29]:
m = market.US_1871_Monthly()

In [38]:
start_year = 1877
length = 30
last_year = 2019-length

start_date = datetime.date(start_year, 1, 1)
last_date = datetime.date(last_year, 12, 1)

def run(stock_pct=Decimal('.6')):
    swrs = {}
    date = start_date
    ic(date, last_date)
    while date <= last_date:
        returns = []
        for annual_returns in m.iter_from(date, length=length*12):
            stocks = annual_returns.stocks - annual_returns.inflation
            bonds = annual_returns.bonds - annual_returns.inflation
            returns.append((stocks * stock_pct) + (bonds * (1-stock_pct)))

        monthly_swr = float(metrics.ssr(returns))
        annual_swr = ((1 + monthly_swr) ** 12) - 1
        swrs[date] = annual_swr
        
        date += relativedelta(months=1)
        
    return pandas.Series(data=swrs)

In [39]:
swr = run()
swr.head()

ic| date: datetime.date(1877, 1, 1)
    last_date: datetime.date(1989, 12, 1)


1877-01-01    0.107117
1877-02-01    0.109876
1877-03-01    0.111214
1877-04-01    0.109739
1877-05-01    0.118524
dtype: float64

In [40]:
df = pandas.DataFrame(data={'swr': swr, 'cape': shiller['CAPE10']}).dropna()
df.head()

,swr,cape
1881-01-01,0.076014,18.47
1881-02-01,0.072636,18.15
1881-03-01,0.073629,18.27
1881-04-01,0.073213,17.95
1881-05-01,0.074203,18.87


In [41]:
df.corr()

,swr,cape
swr,1.000000,-0.765568
cape,-0.765568,1.000000


In [46]:
ols("cape ~ swr", df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   cape   R-squared:                       0.586
Model:                            OLS   Adj. R-squared:                  0.586
Method:                 Least Squares   F-statistic:                     1849.
Date:                Tue, 15 Dec 2020   Prob (F-statistic):          1.98e-252
Time:                        12:05:19   Log-Likelihood:                -3272.1
No. Observations:                1308   AIC:                             6548.
Df Residuals:                    1306   BIC:                             6559.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     29.1497      0.349     83.640      0.000      28.466      29.833
swr         -218.8799      5.090    -43.004      0.000    -228.865    -208.895
==============================================================================
Omnibus:                        3.023   Durbin-Watson:                   0.052
Prob(Omnibus):                  0.221   Jarque-Bera (JB):                3.019
Skew:                           0.073   Prob(JB):                        0.221
Kurtosis:                       3.185   Cond. No.                         62.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [59]:
df['success'] = (df['swr'] < 0.06).astype(int)

In [60]:
ols("cape ~ success", df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   cape   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                     676.7
Date:                Tue, 15 Dec 2020   Prob (F-statistic):          1.52e-120
Time:                        12:09:47   Log-Likelihood:                -3576.0
No. Observations:                1308   AIC:                             7156.
Df Residuals:                    1306   BIC:                             7166.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     12.2018      0.138     88.564      0.000      11.931      12.472
success        5.4007      0.208     26.013      0.000       4.993       5.808
==============================================================================
Omnibus:                       26.296   Durbin-Watson:                   0.092
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               18.879
Skew:                           0.186   Prob(JB):                     7.95e-05
Kurtosis:                       2.543   Cond. No.                         2.50
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""